In [ ]:
"""
NOT MY WORK
This notebook was created by Milàn Simon, 
you can find it here : https://www.linkedin.com/posts/milan-simon_greeks-activity-7298018675196854274-kZCM?utm_source=share&utm_medium=member_desktop&rcm=ACoAAD3P3JoB1XEV8FhZcJeQjrYoV7O1-UjUw6g
"""


# %% Notebook Setup
from IPython.display import clear_output, display
clear_output(wait=True)
%matplotlib inline

# %% Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
import ipywidgets as widgets

# %% Black-Scholes formula definitions (Delta, Gamma, Vega only)
def black_scholes(S, K, T, r, sigma, option_type):
    """
    Compute Delta, Gamma, and Vega using the Black-Scholes formula.

    Parameters:
        S : Underlying asset price
        K : Strike price
        T : Time to expiration (in years)
        r : Risk-free rate
        sigma : Volatility
        option_type : 'Call' or 'Put'

    Returns:
        delta, gamma, vega
    """
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)

    # --- Delta ---
    if option_type == 'Call':
        delta = norm.cdf(d1)
    else:
        delta = -norm.cdf(-d1)

    # --- Gamma ---
    gamma = norm.pdf(d1) / (S * sigma * np.sqrt(T))

    # --- Vega ---
    vega = S * norm.pdf(d1) * np.sqrt(T)

    return delta, gamma, vega

# %% Set up interactive widgets
style = {'description_width': '150px'}
layout = widgets.Layout(width='350px')

S_slider = widgets.FloatSlider(value=100, min=50, max=150, step=5,
                               description='Underlying Price (S)', style=style, layout=layout)
K_slider = widgets.FloatSlider(value=100, min=50, max=150, step=5,
                               description='Strike Price (K)', style=style, layout=layout)
T_slider = widgets.FloatSlider(value=1, min=0.1, max=5, step=0.1,
                               description='Maturity (T)', style=style, layout=layout)
r_slider = widgets.FloatSlider(value=0.05, min=0, max=0.2, step=0.01,
                               description='Risk-free Rate (r)', style=style, layout=layout)
sigma_slider = widgets.FloatSlider(value=0.2, min=0.1, max=0.8, step=0.01,
                                   description='Volatility (σ)', style=style, layout=layout)
option_dropdown = widgets.Dropdown(options=['Call', 'Put'], value='Call',
                                   description='Option Type', style=style, layout=layout)

# %% Create an Output widget to display the plot
plot_output = widgets.Output()

# %% Dynamic update function: all curves vs. Underlying Price S
# %% Dynamic update function: all curves vs. Underlying Price S
def update_plots(S, K, T, r, sigma, option_type):
    """
    Plot Delta, Gamma, and Vega vs. Underlying Price (S) with customized styling.
    """
    # Create a new figure with 1 row and 3 columns
    fig, axs = plt.subplots(1, 3, figsize=(18, 5))
    plt.subplots_adjust(hspace=0.3, wspace=0.3)

    # Set black background for the figure
    fig.patch.set_facecolor('black')

    # We'll vary S from the slider's min to max
    s_values = np.linspace(S_slider.min, S_slider.max, 200)

    # Calculate the Greeks for each s in s_values
    delta_vals = []
    gamma_vals = []
    vega_vals = []
    for s_val in s_values:
        d, g, v = black_scholes(s_val, K, T, r, sigma, option_type)
        delta_vals.append(d)
        gamma_vals.append(g)
        vega_vals.append(v)

    # Current Greeks at the actual S
    current_delta, current_gamma, current_vega = black_scholes(S, K, T, r, sigma, option_type)

    # Styling parameters
    text_color = 'white'
    line_colors = {
        'Delta': 'cyan' if option_type == "Call" else 'blue',
        'Gamma': 'magenta',
        'Vega': 'orange'
    }

    # 1. Delta vs. S
    axs[0].plot(s_values,
                delta_vals,
                lw=2,
                label=f"Delta ({option_type})",
                color=line_colors['Delta'])

    axs[0].scatter(S,
                   current_delta,
                   color='red',
                   zorder=5,
                   label="Current Value")

    axs[0].axhline(0.0,
                   color="gray",
                   linestyle="--",
                   lw=1)

    axs[0].set_title("Delta", color=text_color)
    axs[0].set_xlabel("Underlying price", color=text_color)
    axs[0].set_ylabel("Delta", color=text_color)
    axs[0].legend(facecolor='black', edgecolor='white', labelcolor=text_color)

    # Remove grid and set black background for axes
    axs[0].grid(False)
    axs[0].set_facecolor('black')

    # Set axis tick colors
    axs[0].tick_params(colors=text_color)

    # 2. Gamma vs. S
    axs[1].plot(s_values,
                gamma_vals,
                lw=2,
                label=f"Gamma ({option_type})",
                color=line_colors['Gamma'])

    axs[1].scatter(S,
                   current_gamma,
                   color='red',
                   zorder=5,
                   label="Current Value")

    axs[1].set_title("Gamma", color=text_color)
    axs[1].set_xlabel("Underlying price", color=text_color)
    axs[1].set_ylabel("Gamma", color=text_color)
    axs[1].legend(facecolor='black', edgecolor='white', labelcolor=text_color)

    # Remove grid and set black background for axes
    axs[1].grid(False)
    axs[1].set_facecolor('black')

    # Set axis tick colors
    axs[1].tick_params(colors=text_color)

    # 3. Vega vs. S
    axs[2].plot(s_values,
                vega_vals,
                lw=2,
                label=f"Vega ({option_type})",
                color=line_colors['Vega'])

    axs[2].scatter(S,
                   current_vega,
                   color='red',
                   zorder=5,
                   label="Current Value")

    axs[2].set_title("Vega", color=text_color)
    axs[2].set_xlabel("Underlying price", color=text_color)
    axs[2].set_ylabel("Vega", color=text_color)
    axs[2].legend(facecolor='black', edgecolor='white', labelcolor=text_color)

    # Remove grid and set black background for axes
    axs[2].grid(False)
    axs[2].set_facecolor('black')

    # Set axis tick colors
    axs[2].tick_params(colors=text_color)

    plt.tight_layout()

# Call this function to apply the customizations during plot updates.
    plt.show()

# %% Button to update the graph
calculate_button = widgets.Button(description="Calculate", button_style='success')

def on_calculate_button_clicked(b):
    with plot_output:
        plot_output.clear_output(wait=True)
        update_plots(
            S_slider.value,
            K_slider.value,
            T_slider.value,
            r_slider.value,
            sigma_slider.value,
            option_dropdown.value
        )

calculate_button.on_click(on_calculate_button_clicked)

# %% Arrange and display the controls and plot output
controls = widgets.VBox([
    S_slider,
    K_slider,
    T_slider,
    r_slider,
    sigma_slider,
    option_dropdown
])
display(controls, calculate_button, plot_output)

# %% Draw the initial plot within the output widget
with plot_output:
    update_plots(
        S_slider.value,
        K_slider.value,
        T_slider.value,
        r_slider.value,
        sigma_slider.value,
        option_dropdown.value
    )

Button(button_style='success', description='Calculate', style=ButtonStyle())

Output()